In [2]:
import tensorflow as tf
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
print(tf.version.VERSION)

from win32gui import FindWindow, SetForegroundWindow, GetClientRect, ClientToScreen
import pyautogui
import cv2

import tkinter as tk
import tkinter
from PIL import Image, ImageTk

from threading import Thread

2.1.0


In [3]:
"""
from win32gui import GetWindowText, EnumWindows
def enum_window_titles():
	#returns all open window class names
	def callback(handle, data):
		titles.append(GetWindowText(handle))
	titles = []
	EnumWindows(callback, None)
	return titles
print(enum_window_titles())
enum_window_titles()
"""

'\nfrom win32gui import GetWindowText, EnumWindows\ndef enum_window_titles():\n\t#returns all open window class names\n\tdef callback(handle, data):\n\t\ttitles.append(GetWindowText(handle))\n\ttitles = []\n\tEnumWindows(callback, None)\n\treturn titles\nprint(enum_window_titles())\nenum_window_titles()\n'

In [4]:
checkpoint_path = "C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol\Model-Grayscale-UNET/weights/saved_model.pb"
checkpoint_dir = os.path.dirname(checkpoint_path)

#WindowClassName = 'Rainbow Six'
WindowClassName = 'J2 - Jupyter Notebook - Google Chrome'

PredImage = None

In [5]:
def screenshot(window_title=None, lenx=512, leny=512):
	if window_title:
		hwnd = FindWindow(None, window_title)
		if hwnd:
			SetForegroundWindow(hwnd)
			x, y, x1, y1 = GetClientRect(hwnd)
			x, y = ClientToScreen(hwnd, (x, y))
			x1, y1 = ClientToScreen(hwnd, (x1 - x, y1 - y))
			# x,y,x1,y1 position and size
			x += int((x1-lenx)/2)
			x1 = lenx
			y += int((y1-leny)/2)
			y1 = leny
			im = pyautogui.screenshot(region=(x, y, x1, y1))
			gray = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2GRAY)
			image = np.array(gray).reshape(-1, lenx, leny, 1)
			return image
		else:
			print('Window not found!')
	else:
		im = pyautogui.screenshot()
		gray = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2GRAY)
		image = np.array(gray).reshape(-1, lenx, leny, 1)
		return image

In [6]:
    def ImCompareGray(Im1, Im2, FigSize=(10,10)):
        f = plt.figure(figsize=FigSize)
        f.add_subplot(1,2, 1)
        plt.imshow(Im1, cmap="gray")
        f.add_subplot(1,2, 2)
        plt.imshow(Im2, cmap="gray")
        plt.show(block=True)

In [7]:
def load_K14():
    #model = create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    #model.summary()
    model = tf.keras.models.load_model(checkpoint_dir)
    #model.summary()

    configproto = tf.compat.v1.ConfigProto() 
    configproto.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=configproto) 
    tf.compat.v1.keras.backend.set_session(sess)
    return model

In [8]:
model = load_K14(
)

In [9]:
def xyz(image):
    a = time.time()
    im_preds = model.predict(image, verbose=1)
    print(time.time()-a)
    image = image.reshape(512, 512, 1)
    im_preds = im_preds.reshape(512, 512, 1)
    #ImCompareGray(image, im_preds)
    return im_preds

In [13]:
PredImage=xyz(screenshot(WindowClassName))

1/1 [==============================] - 3s 3s/sample
2.8968608379364014


In [14]:
ImageTesten = None

In [15]:
def recursive(root):
	global PredImage, ImageTesten
	canvas = tk.Canvas(root, width = 512, height = 512)
	canvas.pack()
	ImageTesten = ImageTk.PhotoImage(Image.fromarray((cv2.cvtColor(PredImage,cv2.COLOR_GRAY2RGB) * 255).astype(np.uint8)))
	canvas.create_image(0,0, anchor=tk.NW, image=ImageTesten)

root = tk.Tk()
recursive(root)
root.after(500, recursive(root))
root.mainloop()

In [ ]:
class myThread1(Thread):
	def __init__(self):
		Thread.__init__(self)
		None

	def run(self):
		def recursive(root):
			global PredImage, ImageTesten
			canvas = tk.Canvas(root, width = 512, height = 512)
			canvas.pack()
			ImageTesten = ImageTk.PhotoImage(Image.fromarray((cv2.cvtColor(PredImage,cv2.COLOR_GRAY2RGB) * 255).astype(np.uint8)))
			canvas.create_image(0,0, anchor=tk.NW, image=ImageTesten)

            
            
		root = tk.Tk()
		recursive(root)
		root.after(500, recursive(root))
		root.mainloop()

In [ ]:
inst = myThread1()

In [ ]:
inst.start()

In [14]:
PredImage = screenshot(WindowClassName)